In [ ]:
# default_exp core

# Core reading utils
> API details.

In [ ]:
# hide
from nbdev.showdoc import *

In [ ]:
# exporti
import math
from dataclasses import dataclass, field
from pathlib import Path, PosixPath
from typing import List

import xarray as xr

root = Path("/luna4/maye")

In [ ]:
# export
def raster_to_xarray(fpath, chunk_scale=2):
    """Read raster image into an xarray.DataArray.

    Using the `chunks` keyword in the open_rasterio method
    activates the return of an out-of-memory virtual array instead
    of the in-memory xarray.DataArray

    fpath: pathlib.Path, str
    chunk_scale:
    """
    fpath = Path(fpath)
    da = xr.open_rasterio(
        fpath, chunks={"x": chunk_scale * 2048, "y": chunk_scale * 1024}
    )
    return da

Using the `chunks` parameter in `open_rasterio` returns a `dask.array` version of `xarray.DataArray`:

In [ ]:
p = root / "dems/ldem_128_jp2.tif"
p.exists()

True

In [ ]:
raster_to_xarray(p)

<xarray.DataArray (band: 1, y: 23040, x: 46080)>
dask.array<open_rasterio-bfe0c80ab4d00cc793037f241354f5fa<this-array>, shape=(1, 23040, 46080), dtype=int16, chunksize=(1, 2048, 4096), chunktype=numpy.ndarray>
Coordinates:
  * band     (band) int64 1
  * y        (y) float64 2.729e+06 2.729e+06 2.729e+06 ... -2.729e+06 -2.729e+06
  * x        (x) float64 -5.458e+06 -5.458e+06 ... 5.458e+06 5.458e+06
Attributes:
    transform:                  (236.9011752, 0.0, -5458203.076608, 0.0, -236...
    crs:                        +proj=eqc +lat_ts=0 +lat_0=0 +lon_0=0 +x_0=0 ...
    res:                        (236.9011752, 236.9011752)
    is_tiled:                   1
    nodatavals:                 (-32768.0,)
    scales:                     (0.5,)
    offsets:                    (1737400.0,)
    AREA_OR_POINT:              Area
    DATA_SET_ID:                "LRO-L-LOLA-4-GDR-V1.0"
    INSTRUMENT_ID:              "LOLA"
    INSTRUMENT_NAME:            "LUNAR ORBITER LASER ALTIMETER"
    PRODUCER_INSTITUTION_NAME:  "GODDARD SPACE FLIGHT CENTER"
    PRODUCT_CREATION_TIME:      2017-09-15
    PRODUCT_ID:                 "LDEM_128"
    START_TIME:                 2009-07-13T17:33:17.246
    STOP_TIME:                  2016-11-29T05:48:19
    TARGET_NAME:                MOON

In [ ]:
# export
def mapcycle_to_xarray(fpath, chunk_scale=2):
    """Read an Diviner map cycle rasterio image into a dask.array.


    fpath: str, pathlib.Path
    chunk_scale: int
        Scaling the chunk
    """
    fpath = Path(fpath)
    da = raster_to_xarray(fpath)
    cycle = int(fpath.name.split("_")[4][:-1])
    da = da.assign_coords(band=[cycle])
    return da.rename({"band": "time"})

In [ ]:
# export
def read_images_into_stacked_array(image_paths, name, chunk_scale=2):
    arrays = [raster_to_xarray(p, chunk_scale) for p in image_paths]
    stack = xr.concat(arrays, "time")
    stack.name = name
    return stack


def get_data_for_pixel(xoff, yoff, ReaderClass, image_paths, name, chunk_scale=1):
    stack = read_images_into_stacked_array(image_paths, name, chunk_scale)
    pix = stack.isel(x=xoff, y=yoff)
    pix = pix.where(pix != -32768)
    img = ReaderClass.from_fpath(image_paths[0])
    pix = pix * img.SCALING_FACTOR + img.OFFSET
    return pix.compute()